# Experiment 1
---------------
Transfer Learning with Keras Resnet50

### Load default libraries and prepare the dataset

In [ ]:
%run ./prepare_dataset.ipynb

In [ ]:
img_width, img_height, img_size, batch_size =  prepareConstants(224 ,224,224,16)
train_data_dir, validation_data_dir = setDirs()
train_generator, validation_generator = prepareDatasets(train_data_dir,validation_data_dir,img_width,img_height,batch_size)
checkData(train_generator,validation_generator)

### Load the ResNet 50 Pretrained model from Keras

In [ ]:
#import inception with pre-trained weights. do not include fully #connected layers
inception_base = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

### Test 1
Add layers to resnet with image net with weights  
We load the ResNet-50 with the ImageNet weights. We remove the top so that we can add our own layer according to the number of our classes. We then add our own layers to complete the model architecture.

In [ ]:
# add a global spatial average pooling layer
x = inception_base.output
x = GlobalAveragePooling2D()(x)

# add a fully-connected layer
x = Dense(512, activation='relu')(x)

# and a fully connected output/classification layer
predictions = Dense(103, activation='softmax')(x)

# create the full network so we can train on it
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

In [ ]:
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

inception_transfer.summary()

### Train Test 1

In [ ]:
start_time = time.time()

with tf.device("/device:GPU:0"):
    history_pretrained = inception_transfer.fit_generator(
                                train_generator,
                                steps_per_epoch=train_generator.samples/train_generator.batch_size ,
                                epochs=5,
                                shuffle = True,
                                verbose = 0,
                                validation_data=validation_generator,
                                validation_steps=validation_generator.samples/validation_generator.batch_size
                                )
    
print("\nTraining completed")

elapsed_time = time.time() - start_time
print("Time:",elapsed_time) 

After the training Test 1 model, this are the results  
val_loss: 0.0089 - val_acc: 0.9969

### Test 2
Add layers to resnet without weights and default pretrained model

In [ ]:
#import inception with pre-trained weights. do not include fully #connected layers
inception_base_vanilla = applications.ResNet50(weights=None, include_top=False)

# add a global spatial average pooling layer
x = inception_base_vanilla.output

x = GlobalAveragePooling2D()(x)

# add a fully-connected layer
x = Dense(512, activation='relu')(x)

# and a fully connected output/classification layer
predictions = Dense(103, activation='softmax')(x)

# create the full network so we can train on it
inception_transfer_vanilla = Model(inputs=inception_base_vanilla.input, outputs=predictions)

In [ ]:
inception_transfer_vanilla.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


### Train Test 2

In [ ]:
start_time = time.time()

with tf.device("/device:GPU:0"):
    history_vanilla = inception_transfer_vanilla.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=5,
    shuffle = True,
    verbose = 0,
    validation_data = validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size)

print("\nTraining completed")

elapsed_time = time.time() - start_time
print("Time:",elapsed_time) 

After the training Test 2 model, this are the results  
val_loss: 0.0973 - val_acc: 0.9745

### Validate the Results

In [ ]:
# summarize history for accuracy validation

plt.plot(history_pretrained.history['val_acc'])
plt.plot(history_vanilla.history['val_acc'])
plt.title('model validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.savefig('../plots/exp1_p1_validation_acc.png')
plt.show()


# summarize history for loss validation
plt.plot(history_pretrained.history['val_loss'])
plt.plot(history_vanilla.history['val_loss'])
plt.title('model validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.savefig('../plots/exp1_p1_validation_loss.png')
plt.show()


# summarize history for accuracy training
plt.plot(history_pretrained.history['acc'])
plt.plot(history_vanilla.history['acc'])
plt.title('model training accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.savefig('../plots/exp1_p1_training_acc.png')
plt.show()



# summarize history for accuracy training
plt.plot(history_pretrained.history['loss'])
plt.plot(history_vanilla.history['loss'])
plt.title('model training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.savefig('../plots/exp1_p1_training_loss.png')
plt.show()



Save the model with the best result

### Save the best model on disk on onnx format and keras format

In [ ]:
model_choose = selectBestResults(history_pretrained,history_vanilla)
if(model_choose == 1):
    saveKerasModel(inception_transfer,"resNet50custom")
    saveStandardModel(inception_transfer,"resNet50custom")
    best_model = inception_transfer
if(model_choose == 2):
    saveKerasModel(inception_transfer_vanilla,"resNet50vanilla")
    saveStandardModel(inception_transfer_vanilla,"resNet50vanilla")
    best_model = inception_transfer_vanilla

In [ ]:
print("\nModel 1: Performance")
print("Validation Accuracy max:",history_pretrained.history['val_acc'][-1])
print("Validation Loss max:",history_pretrained.history['val_loss'][-1])
print("Training Accuracy max:",history_pretrained.history['acc'][-1])
print("Training Loss max:",history_pretrained.history['loss'][-1])

In [ ]:
print("\nModel 2: Performance")
print("Validation Accuracy max:",history_vanilla.history['val_acc'][-1])
print("Validation Loss max:",history_vanilla.history['val_loss'][-1])
print("Training Accuracy max:",history_vanilla.history['acc'][-1])
print("Training Loss max:",history_vanilla.history['loss'][-1])

In [ ]:
# summarize history for accuracy
plt.plot(history_pretrained.history['val_acc'])
plt.plot(history_pretrained.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Validation','Training'],loc = 'upper left')
plt.savefig('../plots/exp1_p1_bestmodel_acc.png')
plt.show()


# summarize history for loss
plt.plot(history_pretrained.history['val_loss'])
plt.plot(history_pretrained.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Validation','Training'],loc = 'upper left')
plt.savefig('../plots/exp1_p1_bestmodel_loss.png')
plt.show()


# See errors for the image predicted

In [ ]:
errors = seeErrors(validation_data_dir,validation_generator,best_model)